In [4]:
%cd /workspaces/course-master-of-AI-

/workspaces/course-master-of-AI-


/root/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
import os
import pandas as pd
import cv2

In [6]:
def mask_to_bbox(mask_path):
    # Load the image
    mask = cv2.imread(mask_path)

    # height, width = mask.shape[:2]

    # Preprocess the image (adjust based on marker type)
    gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]

    # Find contours
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)[0]

    # Assuming a single dominant contour (adjust for multiple markers)
    cnt = cnts[0]

    # Get bounding box coordinates
    x, y, w, h = cv2.boundingRect(cnt)
    xmin = x
    ymin = y
    xmax = x+w
    ymax = y+h
    bbox = [xmin, ymin, xmax, ymax]
    return bbox  # convert_to_yolo_bbox(bbox=bbox)

In [7]:
from PIL import Image
import shutil

def copy_image(source_path, destination_path):
    """Copies an image file with error handling."""
    if not os.path.exists(source_path):
        print(f"Error: Source file not found: {source_path}")
        return

    # Create destination directory if it doesn't exist
    directory = os.path.dirname(destination_path)

    # Create the directory structure if it doesn't exist
    os.makedirs(directory, exist_ok=True)
    
    try:
        shutil.copy2(source_path, destination_path)
        print(f"Copied image: {source_path} -> {destination_path}")
    except PermissionError as e:
        print(f"Error copying image: {e}")

    
def write_to_file(content, destination_path):

    # Extract directory path from the full path
    directory = os.path.dirname(destination_path)

    # Create the directory structure if it doesn't exist
    os.makedirs(directory, exist_ok=True) 
    
    with open(destination_path, "w") as file:
        # Write text to the file
        file.write(content)
        print(f"Successfully wrote \"{content}\" to: {destination_path}")

In [28]:
import random


def convert_to_faster_RCNN_dataset(path_to_images="./dataset/mvtec_anomaly_detection/bottle",
                                   output_path='./dataset/frrcnn'):

    path_ground_truths = os.path.join(path_to_images, 'ground_truth')
    path_images = os.path.join(path_to_images, 'test')

    classes = os.listdir(path_ground_truths)

    # print(classes)

    class_objs = [(c, image, image.replace('.', '_mask.'))
                  for c in classes for image in os.listdir(os.path.join(path_images, c))]
    # print(class_objs)

    random.shuffle(class_objs)

    train_size = int(len(class_objs))

    train_objs = class_objs[:train_size]

    temp = "".join([f"\n  {i}: {item}" for i, item in enumerate(classes)])

    data = f"path: {output_path}\n\nnames:{temp}\n"

    write_to_file(data, f"{output_path}/data.yaml")

    create_label_and_file(output_path, path_ground_truths,
                          path_images, classes, train_objs, '.')


def create_label_and_file(output_path, path_ground_truths, path_images, classes, class_objs, folder_name):
    ids = []
    for o in class_objs:
        source_path = os.path.join(path_images, o[0], o[1])
        target_path = os.path.join(
            output_path, 'images', folder_name, f'{o[0]}_{o[1]}')

        mask_source_path = os.path.join(path_ground_truths, o[0], o[2])
        label_target_path = os.path.join(
            output_path, 'labels', folder_name, f'{o[0]}_{o[1]}'.replace('.png', '.txt'))

        copy_image(source_path, target_path)

        # Load the image
        mask = cv2.imread(mask_source_path)
        height, width = mask.shape[:2]
        bbox = mask_to_bbox(mask_source_path)
        
        ids.append([f'{o[0]}_{o[1]}'.replace('.png', ''),classes.index(o[0]), bbox[0], bbox[1], bbox[2], bbox[3]])

        # content = f"{classes.index(o[0])} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}"
        # write_to_file(content, label_target_path)

    df = pd.DataFrame(ids, columns=['image_name', 'label_index', 'x1', 'y1', 'x2', 'y2'])
    df.to_csv(os.path.join(output_path,'dataset.csv'))

In [29]:
convert_to_faster_RCNN_dataset('/workspaces/course-master-of-AI-/dataset/mvtec_anomaly_detection/bottle', '/workspaces/course-master-of-AI-/dataset/faster_RCNN')

Successfully wrote "path: /workspaces/course-master-of-AI-/dataset/faster_RCNN

names:
  0: broken_large
  1: broken_small
  2: contamination
" to: /workspaces/course-master-of-AI-/dataset/faster_RCNN/data.yaml
Error copying image: [Errno 13] Permission denied: '/workspaces/course-master-of-AI-/dataset/faster_RCNN/images/./contamination_016.png'
Error copying image: [Errno 13] Permission denied: '/workspaces/course-master-of-AI-/dataset/faster_RCNN/images/./broken_small_001.png'
Error copying image: [Errno 13] Permission denied: '/workspaces/course-master-of-AI-/dataset/faster_RCNN/images/./broken_large_007.png'
Error copying image: [Errno 13] Permission denied: '/workspaces/course-master-of-AI-/dataset/faster_RCNN/images/./broken_small_015.png'
Error copying image: [Errno 13] Permission denied: '/workspaces/course-master-of-AI-/dataset/faster_RCNN/images/./broken_small_011.png'
Error copying image: [Errno 13] Permission denied: '/workspaces/course-master-of-AI-/dataset/faster_RCNN/ima